# CADC Astroquery
Astroquery is package that provides a set of tools to query astronomical databases. In this tutorial, we will focus on the CADC package of astroquery, which queries data provided by the Canadian Astronomical Data Centre (http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca ). This tutorial will go from data query to displying an image of a supernova. We will gather images from PansSTARRS DR1g, HST, and CFHT, do a cutout of the area we want, and display the three images.


# Tutorial
This tutorial will go through some of the basic functionalities of the CADC astroquery package. The CADC module can be installed using the command:

`pip install --pre astroquery`

TODO: Explain installation and requirements more


(The `--pre` tag installs the pre-release version which we need because the CADC package is not in the latest release.) More information about astroquery can be found at the github repository https://github.com/astropy/astroquery.

Now, onto the tutorial. The first step is to import the CADC module from astroquery.

In [2]:
from astroquery.cadc import Cadc
cadc = Cadc()

# http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/D2.R.fits[17575:18274,13171:13870]
# ^ area I think is nice!

https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/search/?activateMAQ=true&Plane.position.bounds@Shape1Resolver.value=ALL&Observation.collection=CFHTMEGAPIPE&Observation.instrument.name=MegaPrime&Plane.energy.bandpassName=i.MP9701&Plane.energy.bandpassName=i.MP9702&Plane.energy.bandpassName=i.MP9703&Plane.energy.bandpassName=r.MP9601&Plane.energy.bandpassName=r.MP9602#sortCol=caom2%3APlane.time.bounds.lower&sortDir=dsc&col_1=_checkbox_selector;;;&col_2=caom2%3AObservation.uri;;;&col_3=caom2%3AObservation.observationID;;;&col_4=caom2%3APlane.energy.bandpassName;;;&col_5=caom2%3APlane.position.bounds.cval1;;;&col_6=caom2%3APlane.position.bounds.cval2;;;&col_7=caom2%3AObservation.target.name;;D2;&col_8=caom2%3APlane.time.exposure;;;&col_9=caom2%3APlane.position.resolution;;;&col_10=caom2%3APlane.time.bounds.lower;;;&col_11=caom2%3AObservation.collection;;;&col_12=caom2%3AObservation.instrument.name;;;


## Querying
The first step is to query the CADC database and retreive the data we want. In this case, we are looking for a supernova that is images by both CFHT and HST. The CFHTLS has a goal of ______/ Lets take a look at data in the D2 field (https://www.cfht.hawaii.edu/Science/CFHLS/cfhtlsdeepwidefields.html). This is in the region `10:00:28 02:12:30 2000`, so lets try querying by region for the HST and CFHT.


In [ ]:
from astropy.coordinates import SkyCoord, get_constellation

c = SkyCoord('10h00m28s', '+02d12m30s', frame='icrs')

hst_result = cadc.query_region(c, radius=0.1, collection='HST') # HST short for Hubble Space Telescope
cfht_result = cadc.query_region(c, radius=0.1, collection='CFHT')

get_constellation(c)

In [151]:
# Because the length of the queries are so large, I'm going to use ADQL first
# query_vars = { 'collection':'CFHTMEGAPIPE', 'target_name': 'D2', 'ra': 131.28125, 'dec': 54.3, 'radius': 0.0166666 }
# query = '''SELECT *
# FROM caom2.Plane AS Plane 
# JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID 
# WHERE ( Observation.collection = {collection}
# AND Observation.target_name = {target_name}
# AND  ( Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk' )
# AND  ( INTERSECTS( CIRCLE('ICRS',
# {ra},
# {dec},
# {radius}),
# Plane.position_bounds ) = 1 ) )
# )'''.format(query_vars)

cfht_mega_query = '''SELECT *
FROM caom2.Plane AS Plane 
JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID 
WHERE ( Observation.collection = 'CFHTMEGAPIPE'
AND Observation.observationID  LIKE 'MegaPipe.%'
AND Observation.target_name = 'D2'
AND ( Plane.energy_bandpassName LIKE 'r.%' OR Plane.energy_bandpassName LIKE 'i.%' )
AND  ( Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk' ))''' 

job = cadc.run_query(cfht_mega_query, 'sync')
cfht_mega_results = job.get_results()


In [152]:
columns_subset = [
    'collection',
 'observationID',
 'calibrationLevel',
 'algorithm_name',
 'type',
 'intent',
 'target_name',
 'telescope_name',
 'instrument_name',
 'position_bounds',
 'position_bounds_samples',
 'position_bounds_size',
 'position_resolution',
 'position_sampleSize',
 'position_dimension_naxis1',
 'position_dimension_naxis2',
 'productID',
 'dataProductType',
 'position_bounds_points',
]
cfht_mega_results[columns_subset].show_in_notebook()

idx,collection,observationID,calibrationLevel,algorithm_name,type,intent,target_name,telescope_name,instrument_name,position_bounds,position_bounds_samples,position_bounds_size,position_resolution,position_sampleSize,position_dimension_naxis1,position_dimension_naxis2,productID,dataProductType,position_bounds_points
,,,,,,,,,,deg,deg,deg,arcsec,arcsec,,,,,deg
0,CFHTMEGAPIPE,MegaPipe.300.183,3,MEGAPIPE,OBJECT,science,D2,CFHT 3.6m,MegaPrime,[149.79332506007546 1.242003215047531 149.7932641852664 1.7580179555579172 150.30952192965785 1.7580179626898325 150.30946106702183 1.2420032200852684],[150.30946106702197 1.2420032200852764 2.0 149.79332506007538 1.2420032150475198 1.0 149.79326418526625 1.7580179555579287 1.0 150.30952192965785 1.7580179626898504 1.0 0.0 0.0 0.0],0.7297587493542002,--,0.185768447408928,10000,10000,MegaPipe.300.183.R.MP9602,image,[149.79332506007538 1.2420032150475322 149.79326418526622 1.7580179555579178 150.30952192965785 1.7580179626898342 150.30946106702197 1.242003220085266]
1,CFHTMEGAPIPE,MegaPipe.300.183,3,MEGAPIPE,OBJECT,science,D2,CFHT 3.6m,MegaPrime,[149.79332506007546 1.242003215047531 149.7932641852664 1.7580179555579172 150.30952192965785 1.7580179626898325 150.30946106702183 1.2420032200852684],[150.30946106702197 1.2420032200852764 2.0 149.79332506007538 1.2420032150475198 1.0 149.79326418526625 1.7580179555579287 1.0 150.30952192965785 1.7580179626898504 1.0 0.0 0.0 0.0],0.7297587493542002,--,0.185768447408928,10000,10000,MegaPipe.300.183.R.MP9601,image,[149.79332506007538 1.2420032150475322 149.79326418526622 1.7580179555579178 150.30952192965785 1.7580179626898342 150.30946106702197 1.242003220085266]
2,CFHTMEGAPIPE,MegaPipe.300.183,3,MEGAPIPE,OBJECT,science,D2,CFHT 3.6m,MegaPrime,[149.79332506007546 1.242003215047531 149.7932641852664 1.7580179555579172 150.30952192965785 1.7580179626898325 150.30946106702183 1.2420032200852684],[150.30946106702197 1.2420032200852764 2.0 149.79332506007538 1.2420032150475198 1.0 149.79326418526625 1.7580179555579287 1.0 150.30952192965785 1.7580179626898504 1.0 0.0 0.0 0.0],0.7297587493542002,--,0.185768447408928,10000,10000,MegaPipe.300.183.I.MP9703,image,[149.79332506007538 1.2420032150475322 149.79326418526622 1.7580179555579178 150.30952192965785 1.7580179626898342 150.30946106702197 1.242003220085266]
3,CFHTMEGAPIPE,MegaPipe.300.183,3,MEGAPIPE,OBJECT,science,D2,CFHT 3.6m,MegaPrime,[149.79332506007546 1.242003215047531 149.7932641852664 1.7580179555579172 150.30952192965785 1.7580179626898325 150.30946106702183 1.2420032200852684],[150.30946106702197 1.2420032200852764 2.0 149.79332506007538 1.2420032150475198 1.0 149.79326418526625 1.7580179555579287 1.0 150.30952192965785 1.7580179626898504 1.0 0.0 0.0 0.0],0.7297587493542002,--,0.185768447408928,10000,10000,MegaPipe.300.183.I.MP9702,image,[149.79332506007538 1.2420032150475322 149.79326418526622 1.7580179555579178 150.30952192965785 1.7580179626898342 150.30946106702197 1.242003220085266]
4,CFHTMEGAPIPE,MegaPipe.301.184,3,MEGAPIPE,OBJECT,science,D2,CFHT 3.6m,MegaPrime,[150.33358393952332 1.741998141141863 150.33350273703226 2.2580128788591596 150.8499184620945 2.258012888021311 150.84983727584165 1.7419981482087643],[150.8498372758417 1.741998148208748 2.0 150.33358393952318 1.7419981411418661 1.0 150.33350273703223 2.2580128788591467 1.0 150.84991846209445 2.258012888021311 1.0 0.0 0.0 0.0],0.7297587485685686,--,0.185768447408928,10000,10000,MegaPipe.301.184.R.MP9602,image,[150.33358393952318 1.7419981411418632 150.33350273703226 2.2580128788591622 150.84991846209442 2.2580128880213124 150.8498372758417 1.741998148208765]
5,CFHTMEGAPIPE,MegaPipe.301.184,3,MEGAPIPE,OBJECT,science,D2,CFHT 3.6m,MegaPrime,[150.33358393952332 1.741998141141863 150.33350273703226 2.2580128788591596 150.8499184620945 2.258012888021311 150.84983727584165 1.7419981482087643],[150.8498372758417 1.741998148208748 2.0 150.33358393952318 1.7419981411418661 1.0 150.33350273703223 2.2580128788591467 1.0 150.84991846209445 2.25801

In [154]:
urls = cadc.get_data_urls(cfht_mega_results, include_auxiliaries=False)
# The include_auxillaries will include urls to preview files in addtion to the fits files
#[print(url) for url in urls];
urls

['https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/MegaPipe.300.183.R.MP9602.fits?RUNID=vhbh30ahix9l1g0q',
 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/MegaPipe.300.183.R.MP9601.fits?RUNID=p34xl9v5q1p5t237',
 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/MegaPipe.300.183.I.MP9703.fits?RUNID=o5z9h42q06eecsip',
 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/MegaPipe.300.183.I.MP9702.fits?RUNID=v3axg2fbzadgk52e',
 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/MegaPipe.301.184.R.MP9602.fits?RUNID=cuft87z40fwn9g2h',
 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/MegaPipe.301.184.I.MP9703.fits?RUNID=pm5szzc61l3hq9l6',
 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/MegaPipe.301.184.I.MP9701.fits?RUNID=mgsadk3ipc1g3605',
 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/MegaPipe.301.183.R.MP9602.fits?RUNID=rhlkbchff678gplv',
 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/d

In [ ]:
from astropy.io import fits

with fits.open(urls[5]) as hdu_file:
    hdu_file.info()
    image_data = hdu_file[0].data

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# TODO: Can probably delete
print('Min:', np.nanmin(image_data))
print('Max:', np.nanmax(image_data))
print('Mean:', np.nanmean(image_data))
print('Stdev:', np.nanstd(image_data))

plt.imshow(image_data, vmin=0, vmax=1, cmap='gray')
plt.colorbar()


That looks pretty weird... 
Now, lets try and use a cutout!

In [134]:
# TO do a cutout...

import tempfile
from cadccutout import OpenCADCCutout

cadc_cutout = OpenCADCCutout()
output_file = tempfile.mkstemp(suffix='.fits')
input_file = '/path/to/file.fits'

# Cutouts are in cfitsio format.
cutout_region_string = '[300:800,810:1000]'  # HDU 0 along two axes.

# Needs to have 'append' flag set.  The cutout() method will write out the data.
with open(output_file, 'ab+') as output_writer, open(input_file, 'rb') as input_reader:
    cadc_cutout.cutout(input_reader, cadc_cutout, output_writer, cutout_region_string, 'FITS')

'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/MegaPipe.300.183.R.MP9602.fits?RUNID=vhbh30ahix9l1g0q[0:256, 0:256]'

In [ ]:
from astropy.io import fits
from cadccutout import OpenCADCCutout
import tempfile

cadc_cutout = OpenCADCCutout()
output_file = tempfile.mkstemp(suffix='.fits')
cutout_region_string = '[300:800,810:1000]'  # HDU 0 along two axes.

with open(output_file, 'ab+') as output_writer, open(urls[0], 'rb') as input_reader:
    cadc_cutout(input_reader, output_writer, cutout_region_string, 'FITS')
    
print(output_writer)